In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 
import os
from tqdm import tqdm
import json
import time
import random
import pickle

In [2]:
df=pd.read_csv(r'E:\capstone\code\WLASL-master\WLASL-master\start_kit\data.csv',dtype={'video_id': str})
embedding_data=pd.read_csv(r'E:\capstone\code\WLASL-master\WLASL-master\start_kit\embedding_data.csv')

In [3]:
df.head()

,gloss,video_id,xmin,ymin,xmax,ymax,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id
0,book,69241,385,37,885,720,25.0,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0
1,book,65225,190,25,489,370,25.0,-1,1,1,90,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0
2,book,68011,262,1,652,480,25.0,-1,1,2,110,valencia-asl,train,https://www.youtube.com/watch?v=0UsjUE-TXns,0
3,book,68208,123,19,516,358,25.0,60,1,3,113,lillybauer,train,https://www.youtube.com/watch?v=1QOYOZ3g-aY,0
4,book,68012,95,0,1180,720,25.0,-1,1,4,109,valencia-asl,train,https://www.youtube.com/watch?v=aGtIHKEdCds,0


In [4]:
np.unique(df.gloss[:4900]).shape

(285,)

In [5]:
temp_df=df[:4900]

In [6]:
data={}
for i in tqdm(range(0,17000,500)):
    v_data = pickle.load(open(r'E:\capstone\code\WLASL-master\WLASL-master\start_kit\v_data_bw_{}.pkl'.format(i), "rb"))
    for j in v_data.keys():
        if j in list(temp_df['video_id']) :
            data[j]=v_data[j]

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [02:57<00:00,  5.21s/it]


In [7]:
len(data.keys())

3698

In [8]:
train_df=temp_df[temp_df['split']=="train"]
test_df=temp_df[temp_df['split']=="test"]

In [9]:
train_df.head()

,gloss,video_id,xmin,ymin,xmax,ymax,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id
0,book,69241,385,37,885,720,25.0,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0
1,book,65225,190,25,489,370,25.0,-1,1,1,90,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0
2,book,68011,262,1,652,480,25.0,-1,1,2,110,valencia-asl,train,https://www.youtube.com/watch?v=0UsjUE-TXns,0
3,book,68208,123,19,516,358,25.0,60,1,3,113,lillybauer,train,https://www.youtube.com/watch?v=1QOYOZ3g-aY,0
4,book,68012,95,0,1180,720,25.0,-1,1,4,109,valencia-asl,train,https://www.youtube.com/watch?v=aGtIHKEdCds,0


In [10]:
training_input=[]
training_output=[]
testing_output=[]
testing_input=[]
training_word=[]
testing_word=[]

for i in tqdm(range(train_df.shape[0])):
    if train_df.iloc[i]['gloss'] in list(embedding_data.gloss) and train_df.iloc[i]['video_id'] in data.keys() :
        training_input.append(data[train_df.iloc[i]['video_id']])
        training_output.append(list(embedding_data[embedding_data['gloss']==train_df.iloc[i]['gloss']]))
        training_word.append(train_df.iloc[i]['gloss'])

for i in tqdm(range(test_df['video_id'].shape[0])):
    if test_df.iloc[i]['gloss'] in list(embedding_data.gloss) and test_df.iloc[i]['video_id'] in data.keys():
        testing_input.append(data[test_df.iloc[i]['video_id']])
        testing_output.append(list(embedding_data[embedding_data['gloss']==test_df.iloc[i]['gloss']]))
        testing_word.append(test_df.iloc[i]['gloss'])
    
training_input=np.asarray(training_input)
testing_input=np.asarray(testing_input)
training_output=np.asarray(training_output)
testing_output=np.asarray(testing_output)

100%|███████████████████████████████████████████████████████████████████████████████| 636/636 [00:02<00:00, 276.65it/s]


In [11]:
training_input.shape,testing_input.shape,training_output.shape,testing_output.shape

((2568,), (476,), (2568, 301), (476, 301))

In [12]:
training_input[0].shape

(61, 320, 240)

In [13]:
np.unique(training_word).shape,np.unique(testing_word).shape

((285,), (278,))

In [14]:
for i in np.unique(testing_word):
    if i not in np.unique(training_word):
        print(i)

In [15]:
for i in np.unique(training_word):
    if i not in np.unique(testing_word):
        print(i)

children
clock
divorce
glasses
leave
medicine
show


In [17]:
training_input[1].shape

(30, 320, 240)

In [ ]:
training_input_3d=[]
for i in tqdm(training_input):
    data_3d=[]
    for j in i:
        data_3d.append(np.asarray([j,j,j]))
    
    training_input_3d.append(np.asarray(data_3d)) 
    

 33%|██████████████████████████▍                                                    | 859/2568 [01:16<02:44, 10.39it/s]

In [104]:
max_pooler.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling2d_19 (MaxPooling (None, 64, 48, 1)         0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


(1, 159, 117, 1)